In [1]:
import SimpleGUICS2Pygame.simpleguics2pygame as simplegui 
import random
import math

WIDTH = 1000
HEIGHT = 600
FLOOR = 50
PAD_WIDTH = 140
PAD_HEIGHT = 10
HALF_PAD_WIDTH = PAD_WIDTH/2
paddle_pos = WIDTH/2
ball_pos = [paddle_pos, HEIGHT-65]
paddle_vel = 0
in_play = False
LEVEL_SPEED_FACTOR = 1.1
LEVEL_MAX_SPEED_FACTOR = 2.0

class Ball:
    def __init__(self,radius,ball_pos,ball_vel):
        self.radius = radius
        self.ball_pos = ball_pos
        self.ball_vel = ball_vel
        
    def draw(self,canvas,ball_pos):
        canvas.draw_circle(ball_pos, self.radius, 1,"White", "White")
        #ball_angle = math.pi/2 + random.choice((-1,1)) * 
        #            (math.pi*0.01 + math.pi*0.25 * random.randrange(100)/100)
        #sv = ball_VEL * min((LEVEL_SPEED_FACTOR**level), LEVEL_MAX_SPEED_FACTOR)
        #ball_vel = [sv*math.cos(ball_angle), sv*math.sin(ball_angle)]
        
def draw(canvas):
    global paddle_pos, ball_pos, ball_vel
    ball = Ball(10,[paddle_pos, HEIGHT-(FLOOR+PAD_HEIGHT+3)],[4,4])
    ball.draw(canvas,ball_pos)
    ball.ball_pos[0] += ball.ball_vel[0]
    ball.ball_pos[1] += ball.ball_vel[1]
    
    canvas.draw_line([paddle_pos-HALF_PAD_WIDTH, HEIGHT-FLOOR], 
                     [paddle_pos+HALF_PAD_WIDTH, HEIGHT-FLOOR], PAD_HEIGHT, 'White')
    #paddle move
    if paddle_pos >= HALF_PAD_WIDTH:
        paddle_pos += paddle_vel
    else:
        paddle_pos = HALF_PAD_WIDTH
    if paddle_pos <= WIDTH-HALF_PAD_WIDTH:
        paddle_pos += paddle_vel
    else:
        paddle_pos = WIDTH-HALF_PAD_WIDTH
        
def launch_ball():
    global  ball_pos, ball_vel
    #if lives==0:
    #    new_game()
    #if len(blocks)==0:
    #    new_level()
    #if ball_pos==None:
    ball_angle = math.pi/2 + random.choice((-1,1)) * (math.pi*0.01 + math.pi*0.25 * random.randrange(100)/100)
    sv = ball_vel * min((LEVEL_SPEED_FACTOR**level), LEVEL_MAX_SPEED_FACTOR)
    ball_vel = [sv*math.cos(ball_angle), sv*math.sin(ball_angle)]
        
#keydown : 키보드를 누를 때
def keydown(key):
    global paddle_vel, ball_vel
    if key==simplegui.KEY_MAP["left"]:
        paddle_vel = -6
    if key==simplegui.KEY_MAP["right"]:
        paddle_vel = 6
    if key==simplegui.KEY_MAP["space"]:
        inplay = True
        ball_vel = 4
        launch_ball()
        
#keyup : 값을 입력중이 아닐 때 paddle속도는 0
def keyup(key):
    global paddle_vel
    if key==simplegui.KEY_MAP["left"]:
        paddle_vel = 0
    if key==simplegui.KEY_MAP["right"]:
        paddle_vel = 0
        
        
frame = simplegui.create_frame("Break block", WIDTH, HEIGHT)
frame.set_draw_handler(draw)
frame.set_keydown_handler(keydown)
frame.set_keyup_handler(keyup)
frame.start()

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
import SimpleGUICS2Pygame.simpleguics2pygame as simplegui 
import random
import math


# global constants
SIZE = (320,500)

LIVES = 3

TOP_BORDER = 40
BOTTOM_BORDER = 20

BLOCK_SIZE = (50,15)
BLOCK_INFO = (\
  {"color":"red"	, "points":60},\
  {"color":"red"	, "points":60},\
  {"color":"orange"	, "points":50},\
  {"color":"orange"	, "points":50},\
  {"color":"green"	, "points":40},\
  {"color":"green"	, "points":30},\
  {"color":"yellow"	, "points":20},\
  {"color":"yellow"	, "points":10},\
  )

LEVEL_SPEED_FACTOR = 1.1
LEVEL_MAX_SPEED_FACTOR = 2.0

POINT_MULTIPLIER = 1.1

PADDLE_SIZE = (60,7)
PADDLE_MIN_POS = 0
PADDLE_MAX_POS = SIZE[0]-PADDLE_SIZE[0]
PADDLE_VEL = 8

SHOT_RADIUS = 5
SHOT_VEL = 4

MSG_POS = [10, TOP_BORDER+BLOCK_SIZE[1]*len(BLOCK_INFO)]


# global variables
blocks = None
paddle_pos = None
paddle_vel = None
shot_pos = None
shot_vel = None
lives = None
level = None
points = None
show_help = None


# create info about a block
def block_info(pos, size, points, color):
    return {\
      "pos": tuple(pos),\
      "points": points,\
      "color": color,\
      "pos1": (pos[0]+1, pos[1]+size[1]/2),\
      "pos2": (pos[0]+size[0]-1, pos[1]+size[1]/2),\
      }

# keyboard handlers
def keydown_handler(key):
    if key in KEYDOWN_HANDLERS:
        KEYDOWN_HANDLERS[key]()

def keyup_handler(key):
    if key in KEYUP_HANDLERS:
        KEYUP_HANDLERS[key]()

def move_paddle_left():
    global paddle_vel
    paddle_vel = -PADDLE_VEL
    
def move_paddle_right():
    global paddle_vel
    paddle_vel = PADDLE_VEL
    
def stop_paddle_move():
    global paddle_vel
    paddle_vel = 0
        
def launch_shot():
    global show_help, shot_pos, shot_vel
    show_help = False
    if lives==0:
        new_game()
    if len(blocks)==0:
        new_level()
    if shot_pos==None:
        shot_angle = math.pi/2 + random.choice((-1,1)) * (math.pi*0.01 + math.pi*0.25 * random.randrange(100)/100)
        shot_pos = [SIZE[0]/2, SIZE[1]/2]
        sv = SHOT_VEL * min((LEVEL_SPEED_FACTOR**level), LEVEL_MAX_SPEED_FACTOR)
        shot_vel = [sv*math.cos(shot_angle), sv*math.sin(shot_angle)]

# prepare for a new game
def new_game():
    global lives, level, points
    
    lives = LIVES
    level = 0
    points = 0

    new_level()
    
    
# start a new level
def new_level():
    global level, blocks, paddle_pos, paddle_vel, shot_pos, shot_vel
    
    level += 1
    
    blocks = []
    x0 = (SIZE[0] % BLOCK_SIZE[0])/2
    ncols = SIZE[0]//BLOCK_SIZE[0]
    for row in range(len(BLOCK_INFO)):
        for col in range(ncols):
            blocks.append(block_info(\
              (x0+col*BLOCK_SIZE[0],TOP_BORDER+row*BLOCK_SIZE[1]),\
              BLOCK_SIZE,\
              level*BLOCK_INFO[row]["points"],\
              BLOCK_INFO[row]["color"]\
              ))

    paddle_pos = [(PADDLE_MAX_POS+PADDLE_MIN_POS)/2, SIZE[1]-BOTTOM_BORDER]
    paddle_vel = 0
            
    shot_pos = None
    shot_vel = None

    
# draw handler	
def draw(canvas):
    global paddle_angle, shot_pos, lives, points, point_multiplier
    
    # update paddle position
    if paddle_vel!=0:
        paddle_pos[0] = max(PADDLE_MIN_POS, min(PADDLE_MAX_POS, paddle_pos[0] + paddle_vel))

    # check if shot exists
    if shot_pos!=None:
        # adjust shot for velocity 
        shot_pos[0] += shot_vel[0]
        shot_pos[1] += shot_vel[1]
        
        # check for shot hitting a block
        if shot_pos[1]<=TOP_BORDER+len(BLOCK_INFO)*BLOCK_SIZE[1]: # optimization
            block_to_remove = None
            sx = shot_pos[0]
            sy = shot_pos[1]
            sl = shot_pos[0]-SHOT_RADIUS
            sr = shot_pos[0]+SHOT_RADIUS
            st = shot_pos[1]-SHOT_RADIUS
            sb = shot_pos[1]+SHOT_RADIUS
            svx = shot_vel[0]
            svy = shot_vel[1]
            
            for b in blocks:
                bl = b["pos"][0]
                br = b["pos"][0]+BLOCK_SIZE[0]
                bt = b["pos"][1]
                bb = b["pos"][1]+BLOCK_SIZE[1]
                
                if (bt<st<bb and bl<sx<br and svy<0) or (bt<sb<bb and bl<sx<br and svy>0):
                    # bounce off bottom or top of block
                    shot_vel[1] *= -1
                    block_to_remove = b
                    break
                elif (bl<sr<br and bt<sy<bb and svx>0) or (bl<sl<br and bt<sy<bb and svx<0):
                    # bounce off left or right of block
                    shot_vel[0] *= -1
                    block_to_remove = b
                    break
            if block_to_remove!=None:
                blocks.remove(block_to_remove)
                points += int(point_multiplier*block_to_remove["points"])
                point_multiplier *= POINT_MULTIPLIER
            
        # bounce shot off left/right/top walls
        if (shot_pos[0]-SHOT_RADIUS<=0 and shot_vel[0]<0) or (shot_pos[0]+SHOT_RADIUS>=SIZE[0] and shot_vel[0]>0):
            shot_vel[0] *= -1
        if shot_pos[1]-SHOT_RADIUS<=0 and shot_vel[1]<0:
            shot_vel[1] *= -1
            
        # bounce off paddle
        if paddle_pos[0]<shot_pos[0]<paddle_pos[0]+PADDLE_SIZE[0] and paddle_pos[1]<shot_pos[1]+SHOT_RADIUS<paddle_pos[1]+PADDLE_SIZE[1] and shot_vel[1]>0:
            relpos = (shot_pos[0]-paddle_pos[0]) /  PADDLE_SIZE[0]
            if relpos<0.3 or relpos>0.7:
                shot_vel[0] = -(shot_vel[0]+0.5)
            shot_vel[1] *= -1
            point_multiplier = 1
        
        # check for shot passing paddle
        if shot_pos[1]+SHOT_RADIUS>=SIZE[1]:
            shot_pos = None
            lives -= 1

        # check if level over
        if len(blocks)==0:
            shot_pos = None
            lives += 1
            
    # draw blocks
    for b in blocks:
        canvas.draw_line(b["pos1"], b["pos2"], BLOCK_SIZE[1]-2, b["color"])

    # draw paddle
    paddle_pos1 = (paddle_pos[0], paddle_pos[1]+PADDLE_SIZE[1]/2)
    paddle_pos2 = (paddle_pos[0]+PADDLE_SIZE[0], paddle_pos[1]+PADDLE_SIZE[1]/2)
    canvas.draw_line(paddle_pos1, paddle_pos2, PADDLE_SIZE[1], "white")

    # draw shot if needed
    if shot_pos!=None:
        canvas.draw_circle(shot_pos, SHOT_RADIUS, 1, "white", "white")

    # draw text for score, level, and lives
    s = 18
    c = "white"
    
    t = "Score: "+str(points)
    p = [5,20]
    canvas.draw_text(t, p, s, c)
    
    t = "Level: "+str(level)
    p[0] = (SIZE[0]-frame.get_canvas_textwidth(t,s))/2
    canvas.draw_text(t, p, s, c)

    t = "Lives: "+str(lives)
    p[0] = SIZE[0]-frame.get_canvas_textwidth(t,s)-5
    canvas.draw_text(t, p, s, c)
    
    # draw special messages
    text_info = None
    if show_help:
        text_info = (\
          ("Break Out", 18, "cyan"), \
          ("",14,"white"), \
          ("Score points by hitting blocks with the puck.", 14, "white"), \
          ("Bonus points for hitting multiple blocks in sequence.",14,"white"), \
          ("",14,"white"), \
          ("Clear all the blocks to start a new level", 14, "white"), \
          ("and get an extra life.",14,"white"), \
          ("Block points increase with each level,",14,"white"), \
          ("but the puck goes faster too!",14,"white"), \
          ("",14,"white"), \
          ("Use left and right arrow keys to move your paddle.", 14, "white"), \
          ("",14,"white"), \
          ("Written by Roberto Diaz for Code Skulptor", 14, "white"), \
          ("Based on the classic Breakout game by Atari.", 14, "white"), \
          ("",14,"white"), \
          ("Press SPACE to begin game.", 14, "white"), \
          )
    elif lives==0:
        text_info = (\
          ("Game Over!", 24, "white"),\
          ("", 14, "white"),\
          ("Press SPACEBAR to start new game", 14, "white"),\
          )
    elif shot_pos==None:
        text_info = (\
          ("Press SPACEBAR to continue", 14, "white"),\
          )
    if text_info!=None:
        p = list(MSG_POS)
        for info in text_info:
            t = info[0]
            s = info[1]
            c = info[2]
            p[0] = (SIZE[0]-frame.get_canvas_textwidth(t,s))/2
            p[1] += s*1.25
            canvas.draw_text(t, p, s, c)
    

# define key handler lookup (note: must be after the functions are defined)			
KEYDOWN_HANDLERS = {\
  simplegui.KEY_MAP["left"] :move_paddle_left, \
  simplegui.KEY_MAP["right"]:move_paddle_right, \
  simplegui.KEY_MAP["space"]:launch_shot, \
  }
KEYUP_HANDLERS = {\
  simplegui.KEY_MAP["left"] :stop_paddle_move, \
  simplegui.KEY_MAP["right"]:stop_paddle_move, \
  }

  
# initialize and run
frame = simplegui.create_frame("Breakout!", SIZE[0], SIZE[1], 0)
new_game()
show_help = True
frame.set_keydown_handler(keydown_handler)
frame.set_keyup_handler(keyup_handler)
frame.set_draw_handler(draw)
frame.start()